In [2]:
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from keras_bert import get_custom_objects
from utils import mysqlDatabase
from myBertTools import myBertModel, myTokenizer
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [3]:
def class_predict_fuc(text):
    text = str(text)[:100]
    x1, x2 = MyTokenizer.encode(first=text)
    x1, x2 = np.array([x1]), np.array([x2])
    outcome = model.predict([x1, x2])
    percentage = outcome.max()
    label = label_dic[outcome.argmax()]
    return label, percentage

In [4]:
def multi_label_predict(text):
    text = str(text)[:100]
    x1, x2 = MyTokenizer.encode(first=text)
    x1, x2 = np.array([x1]), np.array([x2])
    outcome = model.predict([x1, x2])
    multi_label_set = []
    label_idx = np.where(outcome >= 0.15)[1]
    for idx in label_idx:
        multi_label_set.append(label_dic[idx])
    return multi_label_set

In [5]:
if __name__ == '__main__':
    pretrained_path = '/Users/jackyfu/Desktop/hwf87_git/bert_wwm/'
    config_path = os.path.join(pretrained_path, 'bert_config.json')
    checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
    vocab_path = os.path.join(pretrained_path, 'vocab.txt')
    model_path = '/Users/jackyfu/Desktop/hwf87_git/Dcard_post_classification/model_output/dcard_post_cls_bert_2.h5'
    dic_path = '/Users/jackyfu/Desktop/hwf87_git/Dcard_post_classification/model_output/dcard_cate_label_dic_2.npy'
    ## Load Model
    MyBertModel = myBertModel(pretrained_path, config_path, checkpoint_path, vocab_path)
    token_dict = MyBertModel.get_token_dict()
    MyTokenizer = myTokenizer(token_dict)
    model = load_model(model_path, custom_objects=get_custom_objects())
    label_dic = np.load(dic_path, allow_pickle=True).item()
    ## predict
    text = '''關於80人以上遠距的信 真是疑問滿滿… 
    1.所以第一週遠距的課要怎麼加簽？
    2.是選課上限80以上的就確定第一週遠距，上限80以下第一週不遠距嗎？還是看選課人數？
    3.那如果看選課人數
    '''
    # label, confidance = class_predict_fuc(text)
    # print('看板預測: ', label, ' 信心水準: ', confidance)
    multi_label_set = multi_label_predict(text)
    print('predict multi-label set: ', multi_label_set)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2021-09-06 01:22:52.870692: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-09-06 01:22:52.870840: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-09-06 01:22:56.483558: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-06 01:22:56.483750: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-09-06 01:22:57.414361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


predict multi-label set:  ['考試', '臺灣大學']


## Performance testing

In [6]:
import yaml
with open('config.yml', 'r') as stream:
        myconfig = yaml.load(stream, Loader=yaml.CLoader)
database_username = myconfig['mysql_database']['database_username']
database_password = myconfig['mysql_database']['database_password']
database_ip       = myconfig['mysql_database']['database_ip']
database_name     = myconfig['mysql_database']['database_name']

MysqlDatabase = mysqlDatabase(database_username, database_password, database_ip, database_name)

In [7]:
sql = '''
SELECT df.name forums_name, dp.*
FROM Bigdata.dcard_posts dp
left join Bigdata.dcard_forums df on dp.forumid = df.id
WHERE 1=1
'''
df = MysqlDatabase.select_table(sql)

Successfully select from Bigdata table


In [8]:
df['text'] = df['title'] + ' ' + df['excerpt'] + ' ' + df['topics']
df_test = df[df.createdAt >= '2021-09-05'][['forums_name', 'id', 'title', 'excerpt', 'topics', 'text']]
df_test['predict_forums_name'] = df_test['text'].progress_apply(multi_label_predict)

  0%|          | 0/886 [00:00<?, ?it/s]

2021-09-06 01:23:26.486803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-09-06 01:23:28.150745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [9]:
df_test.to_excel('sample_output.xlsx', index = False)